## 0. Preperation

- Install latest version of SAHI and detrex:

In [ ]:
!pip install -U sahi
!pip install detectron2
!pip install -e . detrex

In [ ]:
cp -r detrex/demo detrex/detrex 

In [ ]:
import os
os.getcwd()

- Import required modules:

In [ ]:
from detrex.demo.predictors import DefaultPredictor
from sahi.utils.detectron2 import Detectron2TestConstants
from sahi.utils.detectron2 import export_cfg_as_yaml
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction, predict, get_prediction
from sahi.utils.file import download_from_url
from sahi.utils.cv import read_image
from IPython.display import Image
from detectron2.config import LazyConfig, instantiate

In [ ]:
model_path = "/home/detrex/dino_r50_4scale_12ep.pth"
config_path = "/home/detrex/projects/dino/configs/dino-resnet/dino_r50_4scale_12ep.py"

## 1. Standard Inference with a Detectron2 Model

- Instantiate a detection model by defining model weight path, confing path and other parameters:

In [ ]:
detection_model = AutoDetectionModel.from_pretrained(
    model_type='detrex',
    model_path=model_path,
    config_path=config_path,
    confidence_threshold=0.5,
    image_size=640,
    # device='cuda:0',
)

- Perform prediction by feeding the get_prediction function with an image path and a DetectionModel instance:

In [ ]:
result = get_prediction("demo_data/small-vehicles1.jpeg", detection_model)

- Visualize predicted bounding boxes and masks over the original image:

In [ ]:
result.export_visuals(export_dir="demo_data/")

Image("demo_data/prediction_visual.png")

## 2. Sliced Inference with a Detrex Model

- To perform sliced prediction we need to specify slice parameters. In this example we will perform prediction over slices of 256x256 with an overlap ratio of 0.2:

In [ ]:
result = get_sliced_prediction(
    "demo_data/small-vehicles1.jpeg",
    detection_model,
    slice_height = 256,
    slice_width = 256,
    overlap_height_ratio = 0.2,
    overlap_width_ratio = 0.2,
)

- Visualize predicted bounding boxes and masks over the original image:

In [ ]:
result.export_visuals(export_dir="demo_data/")

Image("demo_data/prediction_visual.png")

## 3. Prediction Result

- Predictions are returned as [sahi.prediction.PredictionResult](sahi/prediction.py), you can access the object prediction list as:

In [ ]:
object_prediction_list = result.object_prediction_list

In [ ]:
object_prediction_list[0]

In [ ]:
result.to_coco_annotations()[:3]

- ObjectPrediction's can be converted to [COCO prediction](https://github.com/i008/COCO-dataset-explorer) format:

In [ ]:
result.to_coco_predictions(image_id=1)[:3]

- ObjectPrediction's can be converted to [imantics](https://github.com/jsbroks/imantics) annotation format:

In [ ]:
!pip install -U imantics

In [ ]:
result.to_imantics_annotations()[:3]

## 4. Batch Prediction

- Set model and directory parameters:

In [ ]:
model_device = "cuda:0" 
model_confidence_threshold = 0.5

slice_height = 480
slice_width = 480
overlap_height_ratio = 0.2
overlap_width_ratio = 0.2

source_image_dir = "demo_data/"

model_type='detrex'
model_path = "/home/detrex/dino_r50_4scale_12ep.pth"
config_path = "/home/detrex/projects/dino/configs/dino-resnet/dino_r50_4scale_12ep.py"

detection_model = AutoDetectionModel.from_pretrained(
    model_type='detrex',
    model_path=model_path,
    config_path=config_path,
    confidence_threshold=0.5,
    image_size=640,
    # device='cuda:0',
)

- Perform sliced inference on given folder:

In [ ]:
predict(
    detection_model = detection_model,
    # model_type=model_type,
    # model_path=model_path,
    # model_config_path=config_path,
    model_device=model_device,
    model_confidence_threshold=model_confidence_threshold,
    source=source_image_dir,
    slice_height=slice_height,
    slice_width=slice_width,
    overlap_height_ratio=0.2,
    overlap_width_ratio=0.2,
)